In [ ]:
# TODO: Bibliotecas

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os
from dotenv import load_dotenv
import requests
from urllib.parse import urljoin
from datetime import datetime, timedelta
import io
from io import BytesIO
import base64
from email.mime.image import MIMEImage
import openpyxl
import time
import unicodedata



ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# TODO: Início do timer
start_time = time.time()

print("Iniciando a execução do projeto...")
for i in range (1000000):
    pass
        


#----------------------------------------------------------------------------------------------- 
# TODO: Carregar as variáveis de ambiente
load_dotenv(override=True)


api_username = os.getenv('API_USERNAME')
api_password = os.getenv('API_PASSWORD')
api_url = os.getenv("API_URL")

# TODO: Configurações do e-mail
enviar_para = os.getenv('DESTINATARIO')
print(f"Valor original do DESTINATARIO: {enviar_para}")
if enviar_para:
    enviar_para = [email.strip() for email in enviar_para.split(',')]
else:
    enviar_para = []
username_email = os.getenv('EMAIL_USERNAME')
password_email = os.getenv('EMAIL_PASSWORD')
server_email = os.getenv('EMAIL_SERVER')
port_email = int(os.getenv('EMAIL_PORT'))


# TODO: variáveis de período
mes_atual = datetime.now().month - 0 
ano_atual = datetime.now().year
ano_anterior = datetime.now().year - 1
proximas_duas_semanas = (datetime.now()+timedelta(days=15)).strftime('%Y-%m-%d')
duas_ultimas_semanas = (datetime.now()-timedelta(days=15))
ultima_semana = (datetime.now() - timedelta(days=7))
proxima_semana = (datetime.now()+timedelta(days=7))
amanha = (datetime.now()+timedelta(days=0))
ontem = (datetime.now()-timedelta(days=1))

In [32]:
def extrair_ultima_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[-1]


def extrair_segunda_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[1]
        
def extrair_terceira_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[2] if len(values_list) > 2 else None

        
def extrair_apelido_protocolo(x):
    if x is None:
        return None
    elif 'apelido_protocolo' in x:
        return x['apelido_protocolo']
    else:
        return None
    
    
def extrair_campo(x, *chaves):
# """
# Extrai um campo aninhado de um dicionário dado um conjunto de chaves.
# :param x: O dicionário de entrada.
# :param chaves: Chaves para navegar no dicionário.
# :return: O valor extraído ou None se a navegação falhar.
# """
    if isinstance(x, dict):
        for chave in chaves:
            x = x.get(chave)
            if x is None:
                return None
        return x
    return None

#------------------------------------------SEÇÃO--------------------------------------
# TODO: API 
# Corpo do login a ser utilizado no acesso
body = {
    "nome": api_username,
    "password":api_password
}

# Obtençao do token de acesso à polotrial
auth_url = urljoin(api_url, "/sessions")

response = requests.post(auth_url, json = body)

# # Verificar a resposta
# print(f"Status Code: {response.status_code}")
# print(f"Headers: {response.headers}")
# print(f"Content: {response.text}")

# Extraindo o token
token = response.json()["token"]

# Incorporando a string Bearer para inserir
if token:
    auth_token = "Bearer " + token
else:
    print("Falha ao obter o token.")
    
headers = {"Authorization": auth_token}

In [33]:
link = 'https://api.polotrial.com/protocolo?nested=true'

In [34]:
#TODO: Acesso Participantes_visita (Agenda)
rota_protocolo = link.replace('$idProtocol', '1182')
df_protocolo = requests.get(rota_protocolo, headers=headers).json()
df_protocolo = pd.DataFrame(df_protocolo)

In [35]:
dim_protocolo = df_protocolo.copy()


In [36]:
colunas = ['dados_pi', 'dados_tipo_de_iniciativa', 'dados_patrocinador', 'dados_cro_responsavel', 'dados_status_protocolo']

for coluna in colunas:
    dim_protocolo[coluna] = dim_protocolo[coluna].apply(extrair_ultima_informacao)
    
dim_protocolo=dim_protocolo[['id', 'apelido_protocolo', 'dados_pi', 'dados_tipo_de_iniciativa', 'dados_patrocinador', 'dados_cro_responsavel', 'dados_status_protocolo']]

dim_protocolo.head()

,id,apelido_protocolo,dados_pi,dados_tipo_de_iniciativa,dados_patrocinador,dados_cro_responsavel,dados_status_protocolo
0,4,BTK,Dr. Carlos Augusto Quadros,Patrocinador,"Sorrento Therapeutics, Inc.",Synova,Concluído
1,5,BTK,Dra. Caroline Cândida Carvalho de Oliveira,Patrocinador,"Sorrento Therapeutics, Inc.",Synova,Concluído
2,8,RIGEL-FOCUS,Dr. Paulo Roberto Rizzo Genestreti,Patrocinador,"Rigel Pharmaceuticals, Inc",Worldwide Clinical Trials,Concluído
3,9,BIOTEST-998,Dra. Valéria Cristina Aguiar,Patrocinador,Biotest AG,IQVIA,Concluído
4,10,GARDEN,Dr. Eduardo Ramacciotti,Patrocinador,ACTICOR BIOTECH,Clinergy Health,Concluído


In [ ]:
# # Gráfico de barras n° de protocolos x patrocinador
# # contagem = dim_protocolo['dados_patrocinador'].value_counts().sort_values(ascending=True)
# fig = px.bar(
#     data_frame = dim_protocolo,
#     y = dim_protocolo['dados_patrocinador'].value_counts().sort_values(ascending=True).index,
#     x = dim_protocolo['dados_patrocinador'].value_counts().sort_values(ascending=True).values,
#     title = 'Número de protocolos por patrocinador',
#     labels = {'x' : 'Número de protocolos', 'y' : 'Patrocinador'},
#     template = 'plotly_dark',
#     orientation = 'h'
# )
# fig.show()